# Libraries

In [1]:
import sys
import os
directory_path = os.path.abspath(os.path.join('..'))
utils_path = os.path.abspath(os.path.join('../utils'))
if directory_path not in sys.path:
    sys.path.append(directory_path)
    sys.path.append(utils_path)

In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from utils.Validator import *
from utils.Combinations import *
from utils.Network import *
pd.set_option('display.max_columns', None)  
sns.set(rc={'figure.figsize':(11.7,8.27)})

# Digraph: User Followers

In [3]:
df_user_follower = pd.read_csv("../data/processed/DiGraph_Followers.csv")
df_user_follower.head()

,UserId,FollowerUserId
0,368,993
1,368,1902
2,368,1950
3,368,3258
4,368,3429


In [4]:
GUF = nx.from_pandas_edgelist(df_user_follower, source = "UserId", target = "FollowerUserId", create_using=nx.DiGraph())
print(nx.info(GUF))

Name: 
Type: DiGraph
Number of nodes: 253893
Number of edges: 765508
Average in degree:   3.0151
Average out degree:   3.0151


In [5]:
pagerank_centrality =nx.pagerank(GUF)
Network.get_top_nodes(pagerank_centrality)

{4333519: 0.002126725378651298,
 87114: 0.00099114808114556,
 4956446: 0.0009757464270521383,
 3006723: 0.0006596276295753494,
 2357264: 0.00046691348480868794}

In [6]:
nx.set_node_attributes(GUF, name='pagerank_cent_followers', values = pagerank_centrality)

# Graph: Team Members

In [7]:
df_user_team = pd.read_csv("../data/processed/Graph_Teams.csv")
df_user_team.head()

,UserId_1,UserId_2,weight
0,10000270,8243312,3
1,10000270,9829581,2
2,1000033,1003122,1
3,1000033,1214207,1
4,1000033,1581982,1


In [8]:
GUT = nx.from_pandas_edgelist(df_user_team, source = "UserId_1", target = "UserId_2",edge_attr=True, create_using=nx.Graph())
print(nx.info(GUT))

Name: 
Type: Graph
Number of nodes: 155730
Number of edges: 196736
Average degree:   2.5266


In [9]:
eigenvector_centrality = nx.eigenvector_centrality(GUT,weight="weight")
Network.get_top_nodes(eigenvector_centrality)

{637722: 0.7050741444838826,
 2360956: 0.7050741444838826,
 43882: 0.017099873206441504,
 43896: 0.016979948853551405,
 43883: 0.0168582338863929}

In [10]:
nx.set_node_attributes(GUT, name='eigenvector_cent_teams', values = eigenvector_centrality)

# Graph: User Forum Messages

In [11]:
df_user_forum_messages = pd.read_csv("../data/processed/Graph_ForumMessages.csv")
df_user_forum_messages.head()

,UserId_1,UserId_2,weight
0,10000014,10030651,7
1,10000014,10135089,1
2,10000014,10275991,4
3,10000014,10352854,1
4,10000014,10454992,1


In [12]:
GUM = nx.from_pandas_edgelist(df_user_forum_messages, source = "UserId_1", target = "UserId_2",edge_attr=True, create_using=nx.Graph())
print(nx.info(GUM))

Name: 
Type: Graph
Number of nodes: 101284
Number of edges: 4306750
Average degree:  85.0430


In [13]:
eigenvector_centrality = nx.eigenvector_centrality(GUM,weight="weight")
Network.get_top_nodes(eigenvector_centrality)

{3012786: 0.3803401654659919,
 3399844: 0.24816810564171174,
 3490494: 0.24719264760265058,
 5274255: 0.20270009941050648,
 4956446: 0.1942832969748979}

In [14]:
nx.set_node_attributes(GUM, name='eigenvector_cent_forums', values = eigenvector_centrality)

# Gephi Export

In [20]:
nx.write_gexf(GUF, r"../data/gephi/GUF.gexf")
nx.write_gexf(GUT, r"../data/gephi/GUT.gexf")
nx.write_gexf(GUM, r"../data/gephi/GUM.gexf")

# To Dataframe

In [15]:
df_GUF = pd.DataFrame.from_dict(dict(GUF.nodes(data=True)), orient='index').reset_index()
df_GUF = df_GUF.rename(columns = {'index':'UserId'})

df_GUT = pd.DataFrame.from_dict(dict(GUT.nodes(data=True)), orient='index').reset_index()
df_GUT = df_GUT.rename(columns = {'index':'UserId'})

df_GUM = pd.DataFrame.from_dict(dict(GUM.nodes(data=True)), orient='index').reset_index()
df_GUM = df_GUM.rename(columns = {'index':'UserId'})

df_GUF["UserId"] = df_GUF["UserId"].astype(str)
df_GUT["UserId"] = df_GUT["UserId"].astype(str)
df_GUM["UserId"] = df_GUM["UserId"].astype(str)

In [16]:
df_users = pd.read_csv(r"../data/interim/Users.csv",dtype=str)
df_users.head()

,UserId,UserName,DisplayName
0,368,antgoldbloom,Anthony Goldbloom
1,381,iguyon,Isabelle
2,389,anandjeyahar,Anand Jeyahar
3,392,jmeynet,Julien Meynet
4,394,m4xl1n,m4xl1n


In [17]:
df_users_net = df_users.merge(df_GUF,how="left",on=["UserId"]).merge(df_GUT,how="left",on=["UserId"]).merge(df_GUM,how="left",on=["UserId"])
df_users_net.head()

,UserId,UserName,DisplayName,pagerank_cent_followers,eigenvector_cent_teams,eigenvector_cent_forums
0,368,antgoldbloom,Anthony Goldbloom,0.000003,9.451808e-26,0.001207
1,381,iguyon,Isabelle,0.000003,NaN,0.000052
2,389,anandjeyahar,Anand Jeyahar,0.000003,1.171515e-26,0.000033
3,392,jmeynet,Julien Meynet,0.000003,NaN,0.000001
4,394,m4xl1n,m4xl1n,NaN,NaN,NaN


In [18]:
Validator.validar_numerico(df_users_net,["pagerank_cent_followers","eigenvector_cent_teams","eigenvector_cent_forums"])

,Variable,n,Missing,%Missing,Negativo,%Negativo,Unico,Media,Min,Max,IQRMAX,3STD,P50,P75,P90,P95,P99,Outlier IQR,%Outlier IQR,Outlier 3SD,%Outlier 3SD,Outlier P90,%Outlier P90,Outlier P95,%Outlier P95,Outlier P99,%Outlier P99
0,pagerank_cent_followers,194147,112354,57.87,0,0.0,37369,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,12103,6.23,181,0.09,7718,3.98,4090,2.11,818,0.42
0,eigenvector_cent_teams,194147,155035,79.85,0,0.0,13350,0.0,0.0,0.71,0.0,0.02,0.0,0.0,0.0,0.0,0.0,9671,4.98,7,0.00,3912,2.01,1956,1.01,392,0.20
0,eigenvector_cent_forums,194147,92863,47.83,0,0.0,63858,0.0,0.0,0.38,0.0,0.01,0.0,0.0,0.0,0.0,0.0,10522,5.42,481,0.25,10129,5.22,5065,2.61,1013,0.52


In [19]:
# Completamos con -1 todos los valores nulos para las 3 metricas
df_users_net.fillna(-1,inplace=True)

In [20]:
#Dataset Networks
df_users_net.to_csv(r"../data/processed/UserNetworksMetrics.csv",index=False)